In [17]:
import pandas as pd
from selenium import webdriver

# 웹브라우저를 자동으로 띄우기 위한 과정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
driver_loc = "../../stddev/addon/chromedriver/chromedriver.exe"

# 브라우저를 띄움
driver = webdriver.Chrome(executable_path = driver_loc, options = options)

# 다음단계로 넘어갈 때 3초기다림
# import time
# time.sleep(3)
driver.implicitly_wait(3)

# 위에서 띄운 브라우저가 targetUrl로 이동함
targetUrl = "https://sparkkorea.com"
driver.get(targetUrl)

# 왜 있는지 모름...
html = driver.page_source

# 자동 접속하고 싶은 버튼의 xpath를 복사해서 변수지정해준다.
quizBtnXpath = '//*[@id="menu-item-382"]/a'
quizBtn = driver.find_element_by_xpath(quizBtnXpath)
quizBtn.click()

# 만약 xpath가 고정적이지 않거나 활용하기 어렵다면 copyselect를 사용해서 해당태그의 id를 클릭하게 한다.
# quizelementId = "menu-item-382"
# driver.find_element_by_id(quizelementId).click()

In [15]:
# 브라우저 내에서 진행할 키보드로 입력받는 액션을 진행하기 위한 과정
from selenium.webdriver.common.keys import Keys

# 브라우저 내에서 진행할 액션을 입력한다.
googleUrl = "https://www.google.co.kr/"

# 구글에 접속한다.
driver.get(googleUrl)

# 구글 검색창의 xpath를 찾아서 검색창에 스마트금융과를 입력한다.
googleSearchXpath = "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input"
googleSearchBox = driver.find_element_by_xpath(googleSearchXpath)
searchKeyword = "스마트금융과"

# 검색창에 입력한 검색어를 검색하기 위한 enter키를 누른다.
googleSearchBox.send_keys(searchKeyword)
googleSearchBox.send_keys(Keys.ENTER)

# 셀레니움을 다 진행한뒤 크롤링을 진행하기 위한 연결단계
finalUrl = driver.current_url
pgSource = driver.page_source
import bs4
bsObj = bs4.BeautifulSoup(pgSource, "html.parser")

In [13]:
# 스파크 코리아에 접속해서 퀴즈정보를 가져오기
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

# 브라우저를 자동실행하기 위한 과정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
driver_loc = "../../stddev/addon/chromedriver/chromedriver.exe"
driver = webdriver.Chrome(executable_path = driver_loc, options = options)

# 최대 3초를 기다린다.
driver.implicitly_wait(3)

# 접속할 주소를 맵핑
targetUrl = "https://sparkkorea.com"

# 웹페이지 이동
driver.get(targetUrl)

# 최소 1초를 기다린다.
import time
time.sleep(1)

try:
    # 접속하고 싶은 퀴즈 링크에 해당하는 xpath를 변수처리해서 자동클릭 설정함
    quizBtnXpath = '//*[@id="menu-item-382"]/a'
    quizBtn = driver.find_element_by_xpath(quizBtnXpath)
    quizBtn.click()
except Exception as e:
    print(e)

# 현재 접속해 있는 브라우저 URL을 크롤링하기 위해 parser를 진행함
fianlUrl = driver.current_url
pgSource = driver.page_source
bsObj = BeautifulSoup(pgSource, "html.parser")

# 뽑아내고 싶은 데이터를 찾음
divTag = bsObj.find("div",{"id" : "id_spark_quiz"})
aTag = divTag.find_all("a")

# 리스트를 초기화함
nameList = []
UrlList = []

# 반복문을 통해 퀴즈제목과 링크를 분리해서 리스트에 담음
for i in range(0, len(aTag)):
    titleTag = aTag[i]
    nameList.append(titleTag.text)  

for i in range(0, len(aTag)):
    hrefTag = aTag[i].attrs["href"]
    UrlList.append(hrefTag)

# 데이터프레임 씌우기
titleList = pd.DataFrame(zip(nameList, UrlList))

# 컬럼명 맵핑하기
titleList.columns = ["퀴즈제목","링크"]

# 출력하기
titleList

In [ ]:
# 만들어낸 데이터프레임을 csv파일로 저장하기
answer = titleList.to_csv("./quiz.csv", encoding="ms949" , index=False)

In [ ]:
# 메일전송 자동화 작업
import smtplib
from email.message import EmailMessage

# gmail 메일 설정
smtp_gmail = smtplib.SMTP('smtp.gmail.com', 587)

# 서버연결을 설정
smtp_gmail.ehlo()

# 연결을 암호화
smtp_gmail.starttls()

# 로그인
userid = "********"
userpw = "********"
smtp_gmail.login(userid, userpw)

# # 테스트 용도로만 사용함. 직접 입력받으면 자동화가 안되기 때문
# import getpass
# pw = getpass.getpass()

# # 비밀번호를 pc에 데이터 파일로 저장
# with open('pw.pickle', 'wb')as handle:
# pickle.dump(pw, handle, protocol = pickle.HIGHEST_PROTOCOL)

# # 비밀번호를 암호화하기 위해서 pickle을 사용함
# import pickle

# # 비밀번호가 들어있는 파일의 데이터를 읽어와서 pwpickle변수에 저장함
# with open('pw.pickle', 'rb')as handle2:
#     pwpickle = pickle.load(handle2)

# 수신받을 이메일이 있는 데이터파일을 불러온다.
emaillist = pd.read_csv("./emaillist.txt")

# 해당 파일의 원하는 컬럼에 있는 데이터만 가져온다.
to = emaillist.이메일.tolist()

# 이메일을 보내기 위한 작업
msg=EmailMessage()

# 제목 입력
msg['Subject']="퀴즈 정보"

# 내용 입력
msg.set_content("퀴즈 내용")

# 첨부할 파일을 불러온 뒤 읽어서 메일에 첨부하는 과정
file2 = 'quiz.csv'
fp2 = open(file2,'rb')
file_data=fp2.read()
msg.add_attachment(file_data, maintype='text', subtype='plain', filename=file2)

# 발신자 정보
msg['From']='*********@gmail.com'

# 수진자 정보
msg['To'] = ",".join(to)

# 메일 전송
smtp_gmail.send_message(msg)
smtp_gmail.close()

In [ ]:
# 메일 자동발신 함수화 하기
def crawlingFunction(id, pw, mailSub, mailContent, isFile=False, fileName=False):
    import pandas as pd
    import smtplib
    from email.message import EmailMessage

    smtp_gmail = smtplib.SMTP('smtp.gmail.com', 587)
    smtp_gmail.ehlo()
    smtp_gmail.starttls()

    userid = id
    userpw = pw
    smtp_gmail.login(userid, userpw)

    emaillist = pd.read_csv(isFile)
    to = emaillist.이메일.tolist()
    msg=EmailMessage()
    msg['Subject']= mailSub
    msg.set_content(mailContent)

    file = fileName
    fp = open(file,'rb')
    file_data=fp.read()
    msg.add_attachment(file_data, maintype='text', subtype='plain', filename=file)
    msg['From']='********@gmail.com'
    msg['To'] = ",".join(to)
    smtp_gmail.send_message(msg)
    smtp_gmail.close()

In [ ]:
# 함수 사용 예시
crawlingFunction(id = "********", pw = "**********", mailSub = "함수연습", mailContent = "연습결과", isFile="./emaillist.txt", fileName='quiz.csv')